#                 Classification On The Hespress News Dataset
<img src="https://miro.medium.com/v2/resize:fit:828/0*-MNlm46aQfTjdCpp">

# Load Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import string
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,StratifiedKFold,train_test_split,cross_validate,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier,plot_tree
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,classification_report
import warnings
from nltk.corpus import stopwords
from textblob import TextBlob
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')

C:\Users\omar mohamed\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\omar mohamed\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('archive\stories_faits-divers.csv')
df2= pd.read_csv('archive\stories_art-et-culture.csv')
df3= pd.read_csv('archive\stories_regions.csv')
df4= pd.read_csv('archive\stories_sport.csv')
df5= pd.read_csv('archive\stories_medias.csv')
df6= pd.read_csv('archive\stories_economie.csv')
df7= pd.read_csv('archive\stories_tamazight.csv')
df8= pd.read_csv('archive\stories_societe.csv')
df9= pd.read_csv('archive\stories_orbites.csv')
df10= pd.read_csv('archive\stories_politique.csv')
df11= pd.read_csv('archive\stories_marocains-du-monde.csv')

In [3]:
#concatunate data
df = df.append([df2,df3,df4,df5,df6,df7,df8,df9,df10,df11],ignore_index=True).drop(['Unnamed: 0','id'],axis=1).sample(frac = 1,ignore_index=True)

In [4]:
#Combining Title and Story Columns
df['story'] = df['title']+' '+df['story']
df.drop(columns='title',inplace=True)

In [5]:
df.head()

,date,author,story,topic
0,الثلاثاء 19 نونبر 2019 - 22:00,هسبريس ـ فاطمة الزهراء صدور,رصيف الصحافة: موجة إعفاءات تعصف بأطر وزارة الت...,medias
1,السبت 22 غشت 2020 - 23:00,إبراهيم مغراوي من مراكش,"إغلاق متجرَين بعد اقتحام الفيروس أسوار ""مرجان""...",regions
2,الأربعاء 08 ماي 2019 - 22:00,هسبريس ـ فاطمة الزهراء صدور,رصيف الصحافة: الجزائر تخصص تعويضا شهريا لمقرّب...,medias
3,السبت 14 يناير 2017 - 15:00,هسبريس - محمد الراجي,العنصر: خريطة الفقر في المملكة تنطبق على مناطق...,tamazight
4,الجمعة 28 غشت 2020 - 09:00,هسبريس - نورالدين إكجان,"""الفيروس"" يتمدد في المغرب .. ""كورونا"" يقتحم مد...",regions


In [6]:
#drop duplicated
df.drop_duplicates(inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10998 entries, 0 to 10999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    10998 non-null  object
 1   author  10998 non-null  object
 2   story   10998 non-null  object
 3   topic   10998 non-null  object
dtypes: object(4)
memory usage: 429.6+ KB


# Preprocessing

* Normalization
* Punctuation Removal
* Stopwords Removal

In [8]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])


df['story'] = df['story'].apply(lambda x:normalize_arabic(x))
df['story'] = df['story'].apply(lambda x:remove_punctuations(x))
df['story'] = df['story'].apply(lambda x:remove_stop_words(x))

In [9]:
df.head()

,date,author,story,topic
0,الثلاثاء 19 نونبر 2019 - 22:00,هسبريس ـ فاطمة الزهراء صدور,رصيف الصحافه موجه اعفاءات تعصف باطر وزاره التر...,medias
1,السبت 22 غشت 2020 - 23:00,إبراهيم مغراوي من مراكش,اغلاق متجرَين اقتحام الفيروس اسوار مرجان اغلقت...,regions
2,الأربعاء 08 ماي 2019 - 22:00,هسبريس ـ فاطمة الزهراء صدور,رصيف الصحافه الجزاءر تخصص تعويضا شهريا لمقرّب ...,medias
3,السبت 14 يناير 2017 - 15:00,هسبريس - محمد الراجي,العنصر خريطه الفقر المملكه تنطبق علي مناطق يقط...,tamazight
4,الجمعة 28 غشت 2020 - 09:00,هسبريس - نورالدين إكجان,الفيروس يتمدد المغرب كورونا يقتحم مدن وقري الج...,regions


# Preper Data

## Label encoder

In [10]:
en = LabelEncoder()
df.topic = en.fit_transform(df.topic)

## spliting the data


In [11]:
X = df['story']
y = df['topic']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

# Training Machine Learning Models

In [12]:
def model(model, X_train, y_train):
    ac = Pipeline([
    ('vect', CountVectorizer(
        tokenizer = nltk.word_tokenize, 
        analyzer = "word")),
    ('tfidf', TfidfTransformer()),
    ('model', model),
    ])
    ac.fit(X_train, y_train)
    return ac

***Logistic Regression***


In [13]:
logit = model(LogisticRegression(), X_train, y_train)
print("Training Accurcy:--> " , accuracy_score(y_train,logit.predict(X_train)))
print("Testing Accurcy:--> " , accuracy_score(y_test,logit.predict(X_test)))

Training Accurcy:-->  0.954421459422596
Testing Accurcy:-->  0.86


In [14]:
print(classification_report(y_test,logit.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.93      0.88       178
           1       0.83      0.87      0.85       207
           2       0.94      0.94      0.94       196
           3       0.87      0.89      0.88       205
           4       0.97      0.89      0.93       209
           5       0.73      0.77      0.75       200
           6       0.77      0.77      0.77       222
           7       0.85      0.80      0.82       211
           8       0.72      0.74      0.73       189
           9       1.00      0.98      0.99       170
          10       0.98      0.92      0.95       213

    accuracy                           0.86      2200
   macro avg       0.86      0.86      0.86      2200
weighted avg       0.86      0.86      0.86      2200



***SVM***

In [15]:
svm = model(SVC(), X_train, y_train)
print("Training Accurcy:--> " , accuracy_score(y_train,svm.predict(X_train)))
print("Testing Accurcy:--> " , accuracy_score(y_test,svm.predict(X_test)))

Training Accurcy:-->  0.9961354853375767
Testing Accurcy:-->  0.855


In [16]:
print(classification_report(y_test,svm.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88       178
           1       0.84      0.86      0.85       207
           2       0.94      0.93      0.94       196
           3       0.91      0.89      0.90       205
           4       0.98      0.87      0.92       209
           5       0.67      0.79      0.72       200
           6       0.77      0.75      0.76       222
           7       0.85      0.77      0.81       211
           8       0.70      0.77      0.73       189
           9       1.00      0.96      0.98       170
          10       0.98      0.92      0.95       213

    accuracy                           0.85      2200
   macro avg       0.86      0.86      0.86      2200
weighted avg       0.86      0.85      0.86      2200



***Random Forest Classifier***


In [17]:
rfor = model(RandomForestClassifier(), X_train, y_train)
print("Training Accurcy:--> " , accuracy_score(y_train,rfor.predict(X_train)))
print("Testing Accurcy:--> " , accuracy_score(y_test,rfor.predict(X_test)))

Training Accurcy:-->  1.0
Testing Accurcy:-->  0.8095454545454546


In [18]:
print(classification_report(y_test,rfor.predict(X_test)))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83       178
           1       0.73      0.88      0.80       207
           2       0.81      0.96      0.88       196
           3       0.85      0.80      0.82       205
           4       0.97      0.83      0.90       209
           5       0.76      0.56      0.64       200
           6       0.68      0.78      0.73       222
           7       0.77      0.73      0.75       211
           8       0.75      0.59      0.66       189
           9       0.97      0.99      0.98       170
          10       0.94      0.89      0.91       213

    accuracy                           0.81      2200
   macro avg       0.82      0.81      0.81      2200
weighted avg       0.81      0.81      0.81      2200



***XGBoost***


In [19]:
xgb = model(XGBClassifier(), X_train, y_train)
print("Training Accurcy:--> " , accuracy_score(y_train,xgb.predict(X_train)))
print("Testing Accurcy:--> " , accuracy_score(y_test,xgb.predict(X_test)))

Training Accurcy:-->  1.0
Testing Accurcy:-->  0.825


In [20]:
print(classification_report(y_test,xgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86       178
           1       0.77      0.82      0.79       207
           2       0.91      0.92      0.91       196
           3       0.82      0.84      0.83       205
           4       0.95      0.89      0.92       209
           5       0.65      0.61      0.63       200
           6       0.75      0.77      0.76       222
           7       0.79      0.74      0.77       211
           8       0.69      0.72      0.70       189
           9       0.98      0.95      0.96       170
          10       0.95      0.97      0.96       213

    accuracy                           0.82      2200
   macro avg       0.83      0.83      0.83      2200
weighted avg       0.83      0.82      0.83      2200



***LightGBM***


In [21]:
lgb = model(LGBMClassifier(), X_train, y_train)
print("Training Accurcy:--> " , accuracy_score(y_train,lgb.predict(X_train)))
print("Testing Accurcy:--> " , accuracy_score(y_test,lgb.predict(X_test)))

Training Accurcy:-->  1.0
Testing Accurcy:-->  0.8345454545454546


In [22]:
print(classification_report(y_test,lgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       178
           1       0.79      0.85      0.82       207
           2       0.91      0.93      0.92       196
           3       0.81      0.84      0.83       205
           4       0.95      0.88      0.91       209
           5       0.67      0.67      0.67       200
           6       0.76      0.75      0.75       222
           7       0.83      0.78      0.80       211
           8       0.75      0.74      0.74       189
           9       0.96      0.95      0.96       170
          10       0.95      0.95      0.95       213

    accuracy                           0.83      2200
   macro avg       0.84      0.84      0.84      2200
weighted avg       0.84      0.83      0.83      2200



## Model Comparison

In [23]:
log_score = pd.Series(accuracy_score(y_test,logit.predict(X_test)))
svc_score = pd.Series(accuracy_score(y_test,svm.predict(X_test)))
rfor_score = pd.Series(accuracy_score(y_test,rfor.predict(X_test)))
xgb_score = pd.Series(accuracy_score(y_test,xgb.predict(X_test)))
lgb_score = pd.Series(accuracy_score(y_test,lgb.predict(X_test)))


models_score= pd.DataFrame(index=['LogisticRegression','SVC','RForest','XGBClassifier','LGBMClassifier'],
                        data= [log_score,svc_score,rfor_score,xgb_score,lgb_score]).rename(columns={0:'accuracy'}).sort_values('accuracy',ascending=False)
models_score.style.background_gradient(cmap='binary')

,accuracy
LogisticRegression,0.860000
SVC,0.855000
LGBMClassifier,0.834545
XGBClassifier,0.825000
RForest,0.809545
